In [94]:
import torch
from pymatgen.core.structure import Structure
from pymatgen.io.ase import AseAtomsAdaptor

from gptff.model.mpredict import ASECalculator

# Load pre-trained gptff model

In [95]:
model_weights = "../pretrained/gptff_v1.pth"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
calculator = ASECalculator(model_weights, device)

# Basic usage

In [96]:
structure = Structure.from_file("data/NaCl.cif")
structure.remove_oxidation_states()
print(f"Composition is: {structure.composition}")

adp = AseAtomsAdaptor()
atoms = adp.get_atoms(structure)

Composition is: Na4 Cl4


## Static calculation

In [84]:
_atoms = atoms.copy()
_atoms.set_calculator(calculator)

energy = _atoms.get_potential_energy() # unit (eV)
forces = _atoms.get_forces() # unit (eV/Å)
stress = _atoms.get_stress() # unit (GPa)

print(f"Energy is: {energy[0]:.4f} eV")
print(f"\nForces on each atom are: \n {forces}")
print(f"\nStresses are: \n {stress}")

Energy is: -27.4335 eV

Forces on each atom are: 
 [[ 4.06289473e-08 -4.88944352e-09 -5.00585884e-08]
 [ 2.38651410e-09  4.83960321e-08 -9.07311914e-09]
 [-3.01297405e-08  1.45519152e-08 -2.26318662e-08]
 [ 6.64294930e-09 -1.36897143e-08 -6.25732355e-09]
 [ 3.07336450e-08 -4.65661287e-08  5.21540642e-08]
 [-4.65661287e-09 -3.72529030e-09  6.51925802e-09]
 [-3.72529030e-08  2.32830644e-08  1.76951289e-08]
 [-4.36557457e-10 -1.72003638e-08  1.21071935e-08]]

Stresses are: 
 [-1.2516518e+00 -1.2516513e+00 -1.2516515e+00  6.3789116e-09
 -5.5669673e-09 -2.3364457e-08]


C:\Users\tengl\AppData\Local\Temp\ipykernel_16380\1908980648.py:2: FutureWarning: Please use atoms.calc = calc
  _atoms.set_calculator(calculator)


## Structure relaxation

### w/ lattice changing

In [85]:
from ase.optimize.fire import FIRE
from ase.constraints import ExpCellFilter

_atoms = atoms.copy()
_atoms.set_calculator(calculator)

constraints = ExpCellFilter(_atoms) 
optimizer = FIRE(constraints)
optimizer.run(fmax=0.01, steps=100)

print(f"\nOriginal lattice length is: \n{adp.get_structure(atoms).lattice.abc}\n")
print(f"Relaxed lattice length is: \n{adp.get_structure(_atoms).lattice.abc}")

      Step     Time          Energy          fmax
FIRE:    0 16:22:44      -27.433544      218.414863
FIRE:    1 16:22:44      -26.694424      659.602471
FIRE:    2 16:22:44      -27.433546      218.416090
FIRE:    3 16:22:44      -26.694424      659.602818


C:\Users\tengl\AppData\Local\Temp\ipykernel_16380\1736387840.py:5: FutureWarning: Please use atoms.calc = calc
  _atoms.set_calculator(calculator)
C:\Users\tengl\AppData\Local\Temp\ipykernel_16380\1736387840.py:7: FutureWarning: Import ExpCellFilter from ase.filters
  constraints = ExpCellFilter(_atoms)


FIRE:    4 16:22:44      -27.468334       85.031574
FIRE:    5 16:22:44      -27.472519       51.619413
FIRE:    6 16:22:44      -27.475029        0.276708
FIRE:    7 16:22:44      -27.475027        0.251448
FIRE:    8 16:22:44      -27.475029        0.200798
FIRE:    9 16:22:44      -27.475025        0.133225
FIRE:   10 16:22:44      -27.475027        0.053372
FIRE:   11 16:22:44      -27.475027        0.031517
FIRE:   12 16:22:45      -27.475029        0.031506
FIRE:   13 16:22:45      -27.475029        0.030380
FIRE:   14 16:22:45      -27.475027        0.027583
FIRE:   15 16:22:45      -27.475029        0.024669
FIRE:   16 16:22:45      -27.475029        0.021775
FIRE:   17 16:22:45      -27.475029        0.017441
FIRE:   18 16:22:45      -27.475029        0.012624
FIRE:   19 16:22:45      -27.475029        0.007861

Original lattice length is: 
(5.58812644, 5.58812644, 5.58812644)

Relaxed lattice length is: 
(5.7056821956261725, 5.7056824731326605, 5.705682598861223)


### w/o lattice changing

In [93]:
from ase.optimize.bfgs import BFGS

_atoms = atoms.copy()
_atoms.set_calculator(calculator)

optimizer = BFGS(_atoms)
optimizer.run(fmax=0.01, steps=100)

print(f"\nOriginal lattice length is: \n{adp.get_structure(atoms).lattice.abc}\n")
print(f"Relaxed lattice length is: \n{adp.get_structure(_atoms).lattice.abc}")

      Step     Time          Energy          fmax
BFGS:    0 16:25:53      -27.433542        0.000000

Original lattice length is: 
(5.58812644, 5.58812644, 5.58812644)

Relaxed lattice length is: 
(5.58812644, 5.58812644, 5.58812644)


C:\Users\tengl\AppData\Local\Temp\ipykernel_16380\2202944646.py:4: FutureWarning: Please use atoms.calc = calc
  _atoms.set_calculator(calculator)


## Molecular dynamics

**Note**: There are many avaliable ensembles implemented in ASE, see https://wiki.fysik.dtu.dk/ase/ase/md.html#module-ase.md for details.

In [97]:
from ase import units
from ase.md.velocitydistribution import MaxwellBoltzmannDistribution, Stationary

timestep = 2 # unit: fs
starting_temperature = 100 # unit: K
taut = 100 * timestep  # unit: fs
taup = 1000 * timestep # unit: fs

MaxwellBoltzmannDistribution(
    atoms, temperature_K=starting_temperature, force_temp=True
)
Stationary(atoms)

### NVT ensemble

In [98]:
from ase.md.nptberendsen import NVTBerendsen

_atoms = atoms.copy()
_atoms.set_calculator(calculator)

temperature = 300
dyn = NVTBerendsen(
    atoms=_atoms,
    timestep=timestep * units.fs,
    temperature_K=temperature,
    taut=taut * units.fs,
    trajectory='data/trajectory_nvt.trj',
    logfile='data/log_nvt.log',
    loginterval=1,
    append_trajectory=False,
)
dyn.run(steps=100)

C:\Users\tengl\AppData\Local\Temp\ipykernel_16380\402209000.py:4: FutureWarning: Please use atoms.calc = calc
  _atoms.set_calculator(calculator)


True

### NPT ensemble

In [99]:
from ase.md.nptberendsen import Inhomogeneous_NPTBerendsen

_atoms = atoms.copy()
_atoms.set_calculator(calculator)

temperature = 300
pressure = 1.01325e-4 # unit: GPa (1 atm)

# NaCl shows a bulk_modulus of 24 GPa
bulk_modulus_au = 24 / 160.2176 # GPa to eV/A^3
compressibility_au = 1 / bulk_modulus_au

dyn = Inhomogeneous_NPTBerendsen(
    atoms=_atoms,
    timestep=timestep * units.fs,
    temperature_K=temperature,
    pressure_au=pressure * units.GPa,
    taut=taut * units.fs,
    taup=taup * units.fs,
    compressibility_au=compressibility_au,
    trajectory='data/trajectory_npt.trj',
    logfile='data/log_npt.log',
    loginterval=1,
)
dyn.run(steps=100)

C:\Users\tengl\AppData\Local\Temp\ipykernel_16380\4054009454.py:4: FutureWarning: Please use atoms.calc = calc
  _atoms.set_calculator(calculator)


True